In [2]:
import pandas as pd
import numpy as np
from citipy import citipy
import time
from datetime import datetime
import requests

from config import weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [4]:
cities = []

for coordinate in lat_lngs:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    if city not in cities:
        cities.append(city)

len(cities)

744

In [5]:
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
       
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | lata
Processing Record 2 of Set 1 | guerrero negro
Processing Record 3 of Set 1 | rikitea
Processing Record 4 of Set 1 | westport
Processing Record 5 of Set 1 | mataura
Processing Record 6 of Set 1 | jamestown
Processing Record 7 of Set 1 | tuktoyaktuk
Processing Record 8 of Set 1 | saint-philippe
Processing Record 9 of Set 1 | port alfred
Processing Record 10 of Set 1 | bandarbeyla
Processing Record 11 of Set 1 | adrar
Processing Record 12 of Set 1 | koslan
Processing Record 13 of Set 1 | punta arenas
Processing Record 14 of Set 1 | berdigestyakh
Processing Record 15 of Set 1 | new norfolk
Processing Record 16 of Set 1 | maniitsoq
Processing Record 17 of Set 1 | valdivia
Processing Record 18 of Set 1 | cabo san lucas
Processing Record 19 of Set 1 | maghama
City not found. Skipping...
Processing Record 20 of Set 1 | sao filipe
Processing Record 21 of Set 1 | vanavara
Processing Record 22 of Set 1

In [6]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Lata,40.1629,-8.3327,55.81,99,100,3.00,overcast clouds
1,Guerrero Negro,27.9769,-114.0611,64.60,56,100,6.89,overcast clouds
2,Rikitea,-23.1203,-134.9692,78.01,80,100,19.51,overcast clouds
3,Westport,41.1415,-73.3579,39.99,95,100,1.99,overcast clouds
4,Mataura,-46.1927,168.8643,69.48,45,42,1.90,scattered clouds
5,Jamestown,42.0970,-79.2353,27.52,49,100,10.00,overcast clouds
6,Tuktoyaktuk,69.4541,-133.0374,-25.60,68,40,10.36,scattered clouds
7,Saint-Philippe,-21.3585,55.7679,76.59,85,100,11.97,light rain
8,Port Alfred,-33.5906,26.8910,70.75,91,100,20.22,overcast clouds
9,Bandarbeyla,9.4942,50.8122,76.71,61,88,18.81,overcast clouds


In [7]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")